### Улучшение baseline-модели

Этапы проекта:

- Этап 1 - Подготовка среды MLflow - shell скрипт и регистрация базовой модели.

- Этап 2 - Исследовательский Анализ Данных (EDA).

- Этап 3 - Генерация Признаков и Обучение Модели.

- Этап 4 - Отбор Признаков и Обучение Модели.

- Этап 5 - Подбор Гиперпараметров и Обучение Финальной Версии Модели.

In [10]:
import json
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import optuna
import os
import pandas as pd
import psycopg
import psycopg2 as psycopg

from autofeat import AutoFeatRegressor
from catboost import CatBoostRegressor
from category_encoders import CatBoostEncoder
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.compose import ColumnTransformer
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)

np.random.seed(42)


#### Этап 1: Регистрация базовой модели

In [35]:
# Пути и названия файлов заданы в виде параметров
PATH_DATA = '../data'

DATA_FILE = 'initial_data.csv'
DATA_FILE_X_TRAIN = 'x_train.csv'
DATA_FILE_Y_TRAIN = 'y_train.csv'
DATA_FILE_X_TEST = 'x_test.csv'
DATA_FILE_Y_TEST = 'y_test.csv'

PATH_MODELS = '../models'
MODEL_FILE = 'base_model.pkl'

RESULTS_DIR = '../results'
RESULTS_FILE = 'cv_res.json'
RESULTS_FILE_TEST = 'test_res.json'

In [2]:
TABLE_NAME = 'real_estate_clean'

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = 'real_estate_project_#2'
RUN_NAME = 'base_model'
REGISTRY_MODEL_NAME = "real_estate_model_base"

RANDOM_STATE = 42

In [3]:
# Загрузка данных
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
        # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

        # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]


df = pd.DataFrame(data, columns=columns) 

In [4]:
# Исходные данные
df.head(2)

,id,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,target
0,0,6220,1965,6,55.717113,37.781120,2.64,84,12,true,9,false,9.9,19.9,1,false,35.099998,9500000
1,1,18012,2001,2,55.794849,37.608013,3.00,97,10,true,7,false,0.0,16.6,1,false,43.000000,13500000


Данные будем делить на тренировочный и тестовый набор. Предполагается использовать кросс-валидацию, для получения лучшей обобщающей способности модели.

In [5]:
# Разделим датасет на тейн и тест
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
        X, y, shuffle=True, test_size=0.25, random_state=RANDOM_STATE)

print(f'Размер тренировочного набора: {X_train.shape}, {y_train.shape}')
print(f'Размер тестового набора: {X_test.shape}, {y_test.shape}')

Размер тренировочного набора: (77619, 17), (77619,)
Размер тестового набора: (25874, 17), (25874,)


Поскольку я не знаю, как грузить модели из разных окружений вирутальной машины, обучим ее заново со старыми параметрами.

In [13]:
# Трансформируем данные
cat_features = X_train.select_dtypes(include='object')
# Выберем бинарные признаки
potential_binary_features = cat_features.nunique() == 2

binary_cat_features = cat_features[potential_binary_features[potential_binary_features].index]
other_cat_features = cat_features[potential_binary_features[~potential_binary_features].index]
num_features = X_train.select_dtypes(['float', 'int'])

preprocessor = ColumnTransformer(
        [
    ('binary', OneHotEncoder(drop='if_binary'), binary_cat_features.columns.tolist()),
    ('cat', CatBoostEncoder(return_df=False), other_cat_features.columns.tolist()),
    ('num', StandardScaler(), num_features.columns.tolist())
        ],
    remainder='drop',
    verbose_feature_names_out=False
    )

model_base = CatBoostRegressor(n_estimators=1000,
                              random_seed=42,
                              verbose=False,
                              loss_function='RMSE')

pipeline = Pipeline(
        [
        ('preprocessor', preprocessor),
        ('model', model_base)
        ]
    )
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('binary',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['has_elevator',
                                                   'is_apartment']),
                                                 ('cat',
                                                  CatBoostEncoder(return_df=False),
                                                  ['building_type_int',
                                                   'studio']),
                                                 ('num', StandardScaler(),
                                                  ['id', 'building_id',
                                                   'build_year', 'latitude',
                                                   'longitude',
                                                   'ceiling_height',
                                                   'flats_count',
                                                   'floors_total', 'floor',
                                                   'kitchen_area',
                                                   'living_area', 'rooms',
                                                   'total_area'])],
                                   verbose_feature_names_out=False)),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x7fa5217099c0>)])

In [17]:
y_pred = pipeline.predict(X_test)
res_test = root_mean_squared_error(y_test, y_pred)
print(f'Результат на тестовой выборке, RMSE: {round(res_test, 3)}')

Результат на тестовой выборке, RMSE: 2166110.324


In [22]:
# Получим результат при обучении с кросс-валидацией
cv_strategy = StratifiedKFold(n_splits=3)
cv_res = cross_validate(
        pipeline,
        X_train,
        y_train,
        cv=cv_strategy,
        n_jobs=-1,
        scoring='neg_root_mean_squared_error'
        )
for key, value in cv_res.items():
        cv_res[key] = round(value.mean(), 3)

print(f'Результат при кросс-валидации, RMSE: {abs(cv_res[key])}')

Результат при кросс-валидации, RMSE: 2199017.785


In [46]:
model_base.get_params()

{'loss_function': 'RMSE',
 'random_seed': 42,
 'verbose': False,
 'n_estimators': 1000}

In [47]:
pipeline.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('binary', OneHotEncoder(drop='if_binary'),
                                    ['has_elevator', 'is_apartment']),
                                   ('cat', CatBoostEncoder(return_df=False),
                                    ['building_type_int', 'studio']),
                                   ('num', StandardScaler(),
                                    ['id', 'building_id', 'build_year', 'latitude',
                                     'longitude', 'ceiling_height', 'flats_count',
                                     'floors_total', 'floor', 'kitchen_area',
                                     'living_area', 'rooms', 'total_area'])],
                     verbose_feature_names_out=False)),
  ('model', <catboost.core.CatBoostRegressor at 0x7fa5217099c0>)],
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('binary', OneHotEncoder(drop='if_binary'),
                                  ['has

In [32]:
# Сохраним результат теста
with open(f'{RESULTS_DIR}/{RESULTS_FILE_TEST}', 'w') as f:
    json.dump(res_test, f)

In [34]:
# Сохраним результат кросс-валидации
with open(f'{RESULTS_DIR}/{RESULTS_FILE}', 'w') as f:
    json.dump(cv_res, f)

In [36]:
# Сохраним разделенные датасеты
X_train.to_csv(f'{PATH_DATA}/{DATA_FILE_X_TRAIN}', index=None)
y_train.to_csv(f'{PATH_DATA}/{DATA_FILE_Y_TRAIN}', index=None)
X_test.to_csv(f'{PATH_DATA}/{DATA_FILE_X_TEST}', index=None)
y_test.to_csv(f'{PATH_DATA}/{DATA_FILE_Y_TEST}', index=None)

# Сохраним исходный датасет
df.to_csv(f'{PATH_DATA}/{DATA_FILE}', index=None)

Далее залогируем исходные данные, модель и метрики.

In [65]:
cb_params = {'model__loss_function': 'RMSE',
             'model__random_seed': 42,
             'model__verbose': False,
             'model__n_estimators': 1000}

res_test = {'RMSE_test': res_test}

In [68]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" 
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") 
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# настройки для логирования в MLFlow
pip_requirements = '../requirements.txt'
signature = mlflow.models.infer_signature(X_test, y_pred)
input_example = X_test[:10]

# В первый раз создадим эксперимент
#experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metrics(cv_res)
    mlflow.log_metrics(res_test)
    mlflow.log_params(cb_params)

    model_info = mlflow.sklearn.log_model( 
        sk_model=pipeline,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example,
        await_registration_for=60,
        pip_requirements=pip_requirements)

Registered model 'real_estate_model_base' already exists. Creating a new version of this model...
2024/10/29 19:58:52 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: real_estate_model_base, version 1
Created version '1' of model 'real_estate_model_base'.


In [ ]:
# run_id с сохраненными артефактами базовой модели
run_id_base = '1fd53fed6e9e463a8ce9b45e2a668890'

#### Этап 2: Исследовательский Анализ Данных (EDA)
На этом этапе ваша задача - провести тщательный исследовательский анализ данных (EDA), чтобы глубже понять особенности и связи в предоставленном наборе данных. В процессе EDA вы должны обратить внимание на три ключевых аспекта, о которых мы говорили в теме 3 курса. Очень важно, чтобы все результаты вашего исследования, включая визуализации, статистический анализ и предварительные выводы, были аккуратно залогированы в MLflow.

Для более организованного исследования предлагаем следующие рекомендуемые шаги:
- Понимание данных: Первоначально ознакомьтесь с данными, изучите типы данных, проверьте наличие пропущенных значений.
- Визуализация данных: Используйте графики и диаграммы для визуализации распределений признаков и возможных взаимосвязей между ними.
- Статистический анализ: Примените статистические методы для изучения центральных тенденций, разброса и корреляций между признаками.
- Предварительные выводы: На основе проведённого анализа сформулируйте предварительные выводы о данных, которые помогут в дальнейшем этапе моделирования.

Помните, что EDA - это итеративный процесс, в котором вы можете возвращаться к предыдущим шагам для дополнительного анализа, если это будет необходимо. Все находки и выводы должны быть чётко зафиксированы и легко доступны для команды проекта.


In [ ]:
# 2.1 Загрузка данных

In [ ]:
# 2.2. Общий обзор датасета

In [ ]:
# 2.3 Анализ признаков для модели

In [ ]:
# 2.4 Анализ целевой переменной

In [ ]:
# 2.4 Анализ целевой переменной в зависимости от различных признаков

In [ ]:
# 2.5 Выводы после EDA

In [ ]:
# 2.6 логирование артефактов в MLflow

#### Этап 3: Генерация Признаков и Обучение Новой Версии Модели
После тщательного исследовательского анализа данных (EDA), вы, скорее всего, сформировали несколько гипотез относительно новых признаков, которые могут улучшить качество вашей модели. На этом этапе, мы предлагаем вам приступить к генерации новых признаков и последующему обучению модели, используя два подхода:

Ручная генерация признаков: Используйте ваше понимание данных и результаты EDA для создания новых признаков.
Автоматическая генерация признаков: Воспользуйтесь библиотеками для автоматической генерации признаков, чтобы облегчить и ускорить этот процесс.
Важно: Для признаков, созданных вручную, рекомендуется использовать объекты sklearn, такие как Pipeline и ColumnTransformer. Это позволит автоматизировать процесс преобразования данных и облегчить поддержку вашего проекта.

После генерации новых признаков, наступает время обучить новую версию вашей модели, используя эти признаки. Не забудьте залогировать все результаты, включая новые признаки, параметры модели и метрики качества, в MLflow для удобства отслеживания изменений и результатов.

Рекомендуемые шаги:

- Определение и генерация новых признаков на основе ваших гипотез.
- Использование библиотек для автоматической генерации признаков, если это применимо.
- Интеграция новых признаков в вашу модель с помощью Pipeline или ColumnTransformer для ручно созданных признаков.
- Обучение новой версии модели с использованием всех доступных признаков.
- Логирование результатов в MLflow для документирования и анализа эффективности новых признаков и модели.

Этот этап проекта критически важен для повышения точности и эффективности вашей модели. Тщательная работа на этом этапе может существенно повлиять на итоговое качество моделирования.


In [ ]:
# 3.1 ручная генерация признаков

In [ ]:
# 3.2 оборачивание всех преобразований в объекты sklearn

In [ ]:
# 3.3 автоматическая генерация признаков

In [ ]:
# 3.4 обучение новой версии модели

In [ ]:
# 3.5 логирование артефактов в MLflow

#### Этап 4: Отбор Признаков и Обучение Новой Версии Модели
Создание новых признаков — это лишь часть работы. Следующий важный шаг — это убедиться в том, что каждый из этих признаков действительно вносит положительный вклад в качество модели. Некоторые признаки могут оказывать отрицательное влияние на модель, поэтому их следует исключить из анализа.

На этом этапе, мы рекомендуем вам применить различные методы отбора признаков для того, чтобы определить и удалить те признаки, которые не улучшают качество вашей модели. Цель этого этапа — максимизировать производительность модели, удалив избыточные или неинформативные признаки.

После тщательного отбора признаков, пора обучить новую версию вашей модели, уже без негативно влияющих на неё признаков. Важно залогировать результаты этого этапа, включая измененный набор признаков, параметры модели и полученные метрики качества, в MLflow для последующего анализа и сравнения.

Рекомендуемые шаги:

- Применение методов отбора признаков для идентификации и исключения признаков, ухудшающих качество модели.
- Анализ влияния каждого признака на модель, чтобы понять, какие из них наиболее ценные.
- Обучение новой версии модели без негативно влияющих признаков.
- Логирование всех изменений и результатов в MLflow, включая конечный набор признаков, параметры модели и метрики качества.

Этот этап не только поможет улучшить качество вашей модели, но и даст глубокое понимание о важности и влиянии отдельных признаков на результаты моделирования.


In [ ]:
# 4.1 Отбор признаков при помощи метода номер 1

In [ ]:
# 4.2 Отбор признаков при помощи метода номер 2

In [ ]:
# 4.3 Анализ отобранных признаков при помощи двух методов и формирование финального списка с признаками для модели

In [ ]:
# 4.4 Обучение новой версии модели

In [ ]:
# 4.5 Логирование всех артефактов в MLflow

### Этап 5 - подбор гиперпараметров и обучение новой версии модели
После того как мы уделили значительное внимание качеству модели через создание и отбор признаков, пришло время для финального штриха — подбора гиперпараметров. Этот этап является ключевым в финальной части проекта второго спринта, где ваша задача — оптимизировать гиперпараметры модели для достижения наилучшего качества.

Рекомендуется подобрать гиперпараметры как минимум двумя различными методами (например, с использованием Grid Search и Random Search), чтобы вы могли сравнить результаты и выбрать наиболее эффективный набор гиперпараметров для вашей модели. После определения оптимальных гиперпараметров, наступает время обучить финальную версию модели, используя ваши новые признаки.

Рекомендуемые шаги:

- Выбор методов для подбора гиперпараметров: Определитесь с методами, которые вы будете использовать для подбора гиперпараметров (например, Grid Search, Random Search, Bayesian Optimization).
- Подбор гиперпараметров: Примените выбранные методы для нахождения оптимальных значений гиперпараметров вашей модели.
- Сравнение результатов: Анализируйте и сравнивайте результаты, полученные различными методами, для определения наилучшего набора гиперпараметров.
- Обучение финальной модели: Используя выбранные гиперпараметры, обучите финальную версию вашей модели на новых признаках.
- Документирование процесса и результатов: Залогируйте все шаги и результаты в MLflow, включая сравнение методов подбора гиперпараметров и характеристики финальной модели.

Этот этап позволит вам максимально улучшить качество вашей модели перед финальной оценкой, предоставив полное понимание важности и влияния гиперпараметров на производительность модели.

In [ ]:
# 5.1 Подбор гиперпарметров при мощи метода номер 1

In [ ]:
# 5.2 Подбор гиперпарметров при мощи метода номер 2

In [ ]:
# 5.3 Формирование списка гиперпараметров для новой модели

In [ ]:
# 5.4 Обуение финальной версии модели

In [ ]:
# 5.5 Логирование артефактов в MLflow